# 📂 Section 1: Setup & Configuration


# Marketing Campaign Analytics — EDA, Segmentation & Insights

**Dataset:** Kaggle Marketing Campaign (customer-level)  
**Goal:** Analyze customer behavior, segment customers (RFM & K-Means), and assess campaign performance to produce consulting-style insights and visuals.

**What this notebook does:**
1. Load & clean data (handles separators, dates, missing values)
2. Feature engineering (Total Spend, Frequency, Campaign metrics)
3. KPI snapshot
4. EDA visuals (spend distribution, spend by demographics, purchases by channel)
5. RFM segmentation (quartiles) + interpretation
6. Optional K-Means clustering on RFM features
7. Campaign effectiveness analysis by segment and demographics
8. Save cleaned dataset & figures

> Run cells top-to-bottom. Figures will be saved to `figures/` for your README and dashboard.


## 📂 Section 1: Setup & Configuration

In [8]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Matplotlib defaults
plt.rcParams['figure.figsize'] = (9, 5)
plt.rcParams['axes.grid'] = True

# Paths
DATA_PATH = "marketing_campaign.csv"  # dataset in same folder
OUTPUT_DIR = "."                      # save outputs in current folder
FIG_DIR = os.path.join(OUTPUT_DIR, "figures")
os.makedirs(FIG_DIR, exist_ok=True)

print("Setup complete!")


Setup complete!


## 1) Load & Clean Data

# 📂 Section 2: Data Loading & Cleaning

## 📂 Section 2: Data Loading & Cleaning

In [10]:

# Load with correct separator
df = pd.read_csv('marketing_campaign.csv', sep=';')

# Clean column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Convert date
if 'dt_customer' in df.columns:
    df['dt_customer'] = pd.to_datetime(df['dt_customer'], errors='coerce')

# Handle missing values (median for income)
if 'income' in df.columns:
    df['income'] = df['income'].fillna(df['income'].median())

# Feature engineering
spend_cols = ['mntwines','mntfruits','mntmeatproducts','mntfishproducts','mntsweetproducts','mntgoldprods']
df['total_spend'] = df[spend_cols].sum(axis=1)

# Frequency of purchases
purchase_cols = ['numdealspurchases','numwebpurchases','numcatalogpurchases','numstorepurchases']
df['frequency'] = df[purchase_cols].sum(axis=1)

# Total campaigns accepted (Cmp1-5), and last response
campaign_cols = ['acceptedcmp1','acceptedcmp2','acceptedcmp3','acceptedcmp4','acceptedcmp5']
existing_campaign_cols = [c for c in campaign_cols if c in df.columns]
df['total_campaigns_accepted'] = df[existing_campaign_cols].sum(axis=1) if existing_campaign_cols else 0
if 'response' not in df.columns:
    df['response'] = 0  # fallback

# Basic sanity checks
print("Rows:", len(df))
print("Columns:", len(df.columns))
print("Sample columns:", df.columns[:10].tolist())
df.head()


FileNotFoundError: [Errno 2] No such file or directory: 'marketing_campaign.csv'

## 2) KPI Snapshot

### 🧪 Feature Engineering

In [ ]:

kpis = {}

kpis['total_customers'] = len(df)
kpis['avg_income'] = float(df['income'].mean()) if 'income' in df.columns else None
kpis['avg_total_spend'] = float(df['total_spend'].mean())
kpis['median_total_spend'] = float(df['total_spend'].median())
kpis['overall_campaign_response_rate'] = float(df['response'].mean()) if 'response' in df.columns else None
kpis['overall_cmp_accepts_rate'] = float((df['total_campaigns_accepted'] > 0).mean())

print("KPI Snapshot:")
for k, v in kpis.items():
    print(f" - {k}: {v}")


# 📂 Section 3: Key Business KPIs

## 3) EDA — Distributions & Relationships

### 🧪 Feature Engineering

In [ ]:

# 3.1 Distribution of total spend
plt.figure()
plt.hist(df['total_spend'], bins=30)
plt.title('Distribution of Total Spend')
plt.xlabel('Total Spend')
plt.ylabel('Count')
plt.tight_layout()
plt.savefig(os.path.join(FIG_DIR, 'dist_total_spend.png'))
plt.show()

# 3.2 Total spend by education (average)
if 'education' in df.columns:
    spend_by_edu = df.groupby('education')['total_spend'].mean().sort_values(ascending=False)
    plt.figure()
    spend_by_edu.plot(kind='bar')
    plt.title('Average Total Spend by Education')
    plt.xlabel('Education')
    plt.ylabel('Average Total Spend')
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR, 'avg_spend_by_education.png'))
    plt.show()

# 3.3 Total spend by marital status (average)
if 'marital_status' in df.columns:
    spend_by_ms = df.groupby('marital_status')['total_spend'].mean().sort_values(ascending=False)
    plt.figure()
    spend_by_ms.plot(kind='bar')
    plt.title('Average Total Spend by Marital Status')
    plt.xlabel('Marital Status')
    plt.ylabel('Average Total Spend')
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR, 'avg_spend_by_marital_status.png'))
    plt.show()

# 3.4 Income vs Total Spend
if 'income' in df.columns:
    plt.figure()
    plt.scatter(df['income'], df['total_spend'], alpha=0.5)
    plt.title('Income vs Total Spend')
    plt.xlabel('Income')
    plt.ylabel('Total Spend')
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR, 'income_vs_total_spend.png'))
    plt.show()

# 3.5 Purchases by channel (average per customer)
channel_cols = ['numwebpurchases','numcatalogpurchases','numstorepurchases']
present_channels = [c for c in channel_cols if c in df.columns]
if present_channels:
    avg_by_channel = df[present_channels].mean().sort_values(ascending=False)
    plt.figure()
    avg_by_channel.plot(kind='bar')
    plt.title('Average Purchases by Channel')
    plt.xlabel('Channel')
    plt.ylabel('Avg Purchases')
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR, 'avg_purchases_by_channel.png'))
    plt.show()


## 4) RFM Segmentation

# 📂 Section 4: Exploratory Data Analysis (EDA)

### 🧹 Data Cleaning & Preprocessing

In [ ]:

# RFM components:
# R = 'recency' (lower is better / more recent)
# F = 'frequency' (higher better)
# M = 'total_spend' (higher better)

# Guard against duplicate bin edges using duplicates='drop'
rfm = df.copy()

# Ensure no negative or NaNs
rfm['recency'] = rfm['recency'].fillna(rfm['recency'].median())
rfm['frequency'] = rfm['frequency'].fillna(0)
rfm['total_spend'] = rfm['total_spend'].fillna(0)

# Create quartile segments
recency_labels = ['Very Recent','Recent','Moderate','Inactive']
frequency_labels = ['Low','Medium','High','Very High']
monetary_labels = ['Low','Medium','High','Very High']

try:
    rfm['recency_segment'] = pd.qcut(rfm['recency'], 4, labels=recency_labels, duplicates='drop')
except Exception as e:
    print("Recency qcut warning:", e)
    rfm['recency_segment'] = pd.qcut(rfm['recency'].rank(method='first'), 4, labels=recency_labels)

try:
    rfm['frequency_segment'] = pd.qcut(rfm['frequency'], 4, labels=frequency_labels, duplicates='drop')
except Exception as e:
    print("Frequency qcut warning:", e)
    rfm['frequency_segment'] = pd.qcut(rfm['frequency'].rank(method='first'), 4, labels=frequency_labels)

try:
    rfm['monetary_segment'] = pd.qcut(rfm['total_spend'], 4, labels=monetary_labels, duplicates='drop')
except Exception as e:
    print("Monetary qcut warning:", e)
    rfm['monetary_segment'] = pd.qcut(rfm['total_spend'].rank(method='first'), 4, labels=monetary_labels)

rfm['rfm_segment'] = rfm['recency_segment'].astype(str) + '-' + rfm['frequency_segment'].astype(str) + '-' + rfm['monetary_segment'].astype(str)

# Segment sizes
seg_counts = rfm['rfm_segment'].value_counts().sort_values(ascending=False)

print("Top RFM segments by count:")
print(seg_counts.head(10))

# Plot segment counts (top 12 for readability)
top_n = seg_counts.head(12)
plt.figure()
top_n.plot(kind='bar')
plt.title('Top RFM Segments by Count')
plt.xlabel('RFM Segment')
plt.ylabel('Customers')
plt.tight_layout()
plt.savefig(os.path.join(FIG_DIR, 'rfm_segment_counts.png'))
plt.show()

# Merge RFM back to main df
df = df.merge(rfm[['id','recency_segment','frequency_segment','monetary_segment','rfm_segment']], on='id', how='left')


## 5) Campaign Effectiveness by Segment & Demographics

### 🧪 Feature Engineering

In [ ]:

# Acceptance rate by RFM segment (using 'response' as last campaign response)
if 'response' in df.columns:
    acc_by_rfm = df.groupby('rfm_segment')['response'].mean().sort_values(ascending=False)
    print("Last-campaign response rate by RFM segment (top 10):")
    print((acc_by_rfm*100).round(2).head(10))

    plt.figure()
    acc_by_rfm.head(12).plot(kind='bar')
    plt.title('Response Rate by RFM Segment (Last Campaign)')
    plt.xlabel('RFM Segment')
    plt.ylabel('Response Rate')
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR, 'response_rate_by_rfm.png'))
    plt.show()

# Acceptance vs demographics
if 'education' in df.columns and 'response' in df.columns:
    acc_by_edu = df.groupby('education')['response'].mean().sort_values(ascending=False)
    plt.figure()
    acc_by_edu.plot(kind='bar')
    plt.title('Response Rate by Education')
    plt.xlabel('Education')
    plt.ylabel('Response Rate')
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR, 'response_rate_by_education.png'))
    plt.show()

if 'marital_status' in df.columns and 'response' in df.columns:
    acc_by_ms = df.groupby('marital_status')['response'].mean().sort_values(ascending=False)
    plt.figure()
    acc_by_ms.plot(kind='bar')
    plt.title('Response Rate by Marital Status')
    plt.xlabel('Marital Status')
    plt.ylabel('Response Rate')
    plt.tight_layout()
    plt.savefig(os.path.join(FIG_DIR, 'response_rate_by_marital_status.png'))
    plt.show()


# 📂 Section 5: Customer Segmentation (RFM Analysis)

## 6) (Optional) K-Means Clustering on RFM

### 🧹 Data Cleaning & Preprocessing

In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Prepare RFM features
X = rfm[['recency','frequency','total_spend']].copy()
X = X.replace([np.inf, -np.inf], np.nan).fillna(0)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Simple 3-cluster solution
kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
rfm['cluster'] = kmeans.fit_predict(X_scaled)

# Profile clusters
cluster_profile = rfm.groupby('cluster')[['recency','frequency','total_spend']].mean().round(2)
print("Cluster profile (means):")
print(cluster_profile)

# Merge cluster back
df = df.merge(rfm[['id','cluster']], on='id', how='left')

# Plot cluster sizes
cluster_counts = rfm['cluster'].value_counts().sort_index()
plt.figure()
cluster_counts.plot(kind='bar')
plt.title('Cluster Sizes (K=3)')
plt.xlabel('Cluster')
plt.ylabel('Customers')
plt.tight_layout()
plt.savefig(os.path.join(FIG_DIR, 'cluster_sizes.png'))
plt.show()


## 7) Auto-Generated Insight Starters

# 📂 Section 6: Conclusions & Business Insights

### 🧪 Feature Engineering

In [ ]:

insights = []

# High-value customers threshold
hi_spend_thresh = df['total_spend'].quantile(0.8)
hi_val_share = (df['total_spend'] >= hi_spend_thresh).mean()

insights.append(f"Top 20% customers account for ~{hi_val_share*100:.1f}% of the base (define tailored retention campaigns).")

# Best responding RFM segment (if computed)
if 'response' in df.columns and 'rfm_segment' in df.columns:
    resp_by_seg = df.groupby('rfm_segment')['response'].mean().sort_values(ascending=False)
    if len(resp_by_seg) > 0:
        top_seg, top_rate = resp_by_seg.index[0], resp_by_seg.iloc[0]*100
        insights.append(f"Best last-campaign response segment: **{top_seg}** at ~{top_rate:.1f}% response. Prioritize this segment for the next campaign.")

# Education / Marital signals
if 'education' in df.columns and 'response' in df.columns:
    best_edu = df.groupby('education')['response'].mean().sort_values(ascending=False).index[0]
    insights.append(f"Highest response by education group: **{best_edu}**. Consider tailored creatives for this cohort.")

if 'marital_status' in df.columns and 'response' in df.columns:
    best_ms = df.groupby('marital_status')['response'].mean().sort_values(ascending=False).index[0]
    insights.append(f"Highest response by marital status: **{best_ms}**.")

for i, s in enumerate(insights, 1):
    print(f"{i}. {s}")


## 8) Save Cleaned Dataset & Notes

### 🧪 Feature Engineering

In [ ]:

clean_path = os.path.join(OUTPUT_DIR, "clean_marketing_campaign.csv")
df.to_csv(clean_path, index=False)
print("Saved cleaned dataset to:", clean_path)
print("Figures saved to:", FIG_DIR)



---

**Notebook generated:** 2025-09-05 13:41:59 UTC  

**Next steps (for portfolio):**
- Add 3–5 bullet insights to your README
- Insert key figures from `figures/`
- Build a Tableau/Power BI dashboard using `clean_marketing_campaign.csv`
- Write a short executive summary like a consultant
